# Information Extraction with LLMs using SageMaker JumpStart


[Amazon SageMaker JumpStart](https://aws.amazon.com/sagemaker/jumpstart/) is a machine learning (ML) hub that can help you accelerate your ML journey. With SageMaker JumpStart, you can evaluate, compare, and select FMs quickly based on pre-defined quality and responsibility metrics to perform tasks like article summarization and image generation. Pretrained models are fully customizable for your use case with your data, and you can easily deploy them into production with the user interface or SDK. In addition, you can access prebuilt solutions to solve common use cases, and share ML artifacts, including ML models and notebooks, within your organization to accelerate ML model building and deployment. Using SageMaker Jumpstart, none of your data is used to train the underlying models. Since all data is encrypted and does not leave your virtual private cloud (VPC), you can trust that your data will remain private and confidential.
With the Low code capabilities that SageMaker JumpStart offers, it's now easier than ever for developers to integrate powerful natural language processing into their applications.

In this series of notebooks, we will walk through examples of building information extraction use cases, combining the power of LLMs with prompt engineering and LLM frameworks such as LangChain. We will also examine the uplift of fine-tuning an LLMs for a specific extractive task.


## Prompt Engineering

Promopt engineering is a technique that enables the user to instruct the large language models to generate suggestions, explanations, or completions of text in an interactive way.

In the following section, we start by domonstration of Prompt engineering techniques that help unlocking the power of large language models that provide helpful constraints and steer the model towards intended behavior with the focus on extractive use cases.

### Use Cases:
They key uses cases covered are:

**- Sensitive Information Detection and Redaction**

**- Key Entity Extraction.** simple and more strutcured Key entity extraction

**- Classification** using prompt engineering and fine-tuning


Before we explore each of these usecases one by one, we need to set up our development environment.

<!-- **Topic Modeling**:

Extracting topic of conversation.
 -->
<!-- Topic modelling is used in a number of usecases, for example; topic modelling
for Document organization where we analyze a large collection of documents or articles to
automatically discover the main themes and topics covered. This allows effective
organization and search.

Topic modeling for content recommendation: aimed to Identify topics of interest for a user
based on their past activity and recommend related content.

Topic modeling for trend analysis - Track topics and trends over time based on data like
social media posts, call center calls, etc.
Insert example and code
As you can see in the above, by using simple prompts we can guide the model towards recognizing
the topic of the conversation, without requiring large amount of data or training a model.
 -->


<!-- There are many usecases that require us to extract a specific topic/ intent of a
conversation or document. For examples for Chatbots , we always need to Classify user
queries to understand intent and provide the right response, like booking a flight or checking order
status.
<!-- 
**(Optional) Generate Label data using LLMs**

- We may don't have examples. But we can explain.
 -->
<!-- **Fine-tuning LLMs** --> 

## Prerequisies and Setup the environment

We start by installing necessary packages. This includes upgrading the SageMaker Python SDK and installing Langchain.

In [ ]:
%pip install --quiet --upgrade sagemaker langchain


In [ ]:
import logging
import warnings
import sagemaker
import utils

# TODO: remove this
# To autoreload the module and incorporate the on-going changes on the file
%load_ext autoreload
%autoreload 1
%aimport utils

# Disable warnings and verbose logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

role_arn = utils.get_role_arn()


# Deploy Llama-70B-Chat using SageMaker Jumpstart

First we need to choose an LLM from SageMaker Jumpstart model hub- In this example we are choosing LLama2-70B-chat, but you may use a different model depending on your usecase. Explore the list of SageMaker Jumpstart models [here](https://sagemaker.readthedocs.io/en/v2.82.0/doc_utils/jumpstart.html). To deploy a model from Jumpstart we can use either APIs and the model ID to deloy the model as demonstrated below, or you can use the UI to do that. Once the model is deployed we do a test by asking a qustion from the model.

#### For the reusability, we have put some of the APIs that we will use throughout these notebook in the <mark>utils.py</mark> library- you can study that for better understanding of all teh details.


In [ ]:
# # Llama 70b chat
from sagemaker.jumpstart.model import JumpStartModel

model_id, model_version = "meta-textgeneration-llama-2-70b-f", "2.*"
endpoint_name = model_id
instance_type = "ml.g5.48xlarge"

model = JumpStartModel(
    model_id=model_id, model_version=model_version, role=role_arn
)
predictor = model.deploy(
    endpoint_name=endpoint_name, instance_type=instance_type
)

In [ ]:
question = "What is the capital of France?"
response = utils.llama2_chat(
    predictor,
    system="You are an expert on geography.",
    user=question,
    temperature=0.1,
    max_tokens=512,
    top_p=0.9,
    system=None,
)


print(utils.llama2_parse_output(response))

## Sensitive Data Extraction and Redaction
LLMs show promise for extracting sensitive information for redaction, but designing effective prompts is key to guiding the models properly. Prompt engineering techniques like priming the model to understand the redaction task and providing examples can improve performance. In real-life applications however, additional evalution is required to increase the reliability and safety of LLMs for handling confidential data. 

In the following you can see few examples of using prompt engineering for extraction and redaction of PIIs.

In [ ]:
report_sample = """
This month at AnyCompany, we have seen a significant surge in orders from a diverse clientele. On November 5th, 2023, customer Alice from US placed an order with total of $2190. Following her, on Nov 7th, Bob from UK ordered a bulk set of twenty-five ergonomic keyboards for his office setup with total of $1000. The trend continued with Jane from Australia, who on Nov 12th requested a shipment of ten high-definition monitors with total of $9000, emphasizing the need for environmentally friendly packaging. On the last day of that month, customer John, located in Singapore, finalized an order for fifteen USB-C docking stations, aiming to equip his design studio with the latest technology for total of $3600.
"""

system = """
Your task is to precisely identify Personally Identifiable Information (PII) and identifiable details, including name, address, and the person's country, in the provided text. Replace these details with exactly four asterisks (****) as the masking characters. Use '****' for masking text of any length. Only write the masked text in the response.
"""

In [ ]:
response = utils.llama2_chat(
    predictor,
    system=system,
    user=report_sample,
)
print(utils.llama2_parse_output(response))

## Entity Extraction
In this approach we use prompt engineering to extract key entities from the text. Entities such as names, places, dates, etc.

Entity extraction is the process of identifying and extracting key information entities from unstructured text. Entity extraction helps create structured data from unstructured text and provides useful contextual information for many downstream natural language processing tasks. Some of the common use cases for entity extractions include Extracting information to build a knowledge base, extract metadata to use for personalization or search as well as within chatbots to improve user inputs and conversations understanding. 


### Extracting named entities in a structured format (simple)

In [ ]:
email_sample = "Hello, My name is John. Your AnyCompany Financial Services, LLC credit card account 1111-0000-1111-0008 has a minimum payment of $24.53 that is due by July 31st. Based on your autopay settings, we will withdraw your payment on the due date from your bank account number XXXXXX1111 with the routing number XXXXX0000. Customer feedback for Sunshine Spa, 123 Main St, Anywhere. Send comments to Alice at alice_aa@anycompany.com and Bob at bob_bb@anycompany.com. I enjoyed visiting the spa. It was very comfortable but it was also very expensive. The amenities were ok but the service made the spa a great experience."

system = """
Your task is to precisely identify any email addresses from the given text and then write them, one per line. Remember to ONLY write an email address if it's precisely spelled out in the input text. If there are no email addresses in the text, write "N/A". DO NOT write anything else.
"""

In [ ]:
result = utils.llama2_chat(predictor, system=system, user=email_sample)
print(utils.llama2_parse_output(result))

### Extracting more complex entities in a structured format
Using the previous sample report, we can extract more complex information in a structured way. This time we will pass on a json template for the model to use and return the output in that json format.

In [ ]:
system = """
Your task is to precisely extract information from the text provided, and format it according to the given JSON schema delimited with triple backticks. Only include the JSON output in your response. If a specific field has no available data, indicate this by writing `null` as the value for that field in the output JSON. In cases where there is no data available at all, return an empty JSON object. Avoid including any other statements in the response.

```
{json_schema}
```
"""

In [ ]:
import json

json_schema = """
{
    "orders":
        [
            {
                "name": "<customer_name>",
                "location": "<customer_location>",
                "order_date": "<order_date in format YYYY-MM-DD>",
                "order_total": "<order_total>",
                "order_items": [
                    {
                        "item_name": "<item_name>",
                        "item_quantity": "<item_quantity>"
                    }
                ]
            }
        ]
}
"""


response = utils.llama2_chat(
    predictor,
    system=system.format(json_schema=json_schema),
    user=report_sample,
)
json_str = utils.llama2_parse_output(response)
print(json_str)

# Use Pydantic with Langhain Output Parsers
[Output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/) from [Langchain](https://www.langchain.com/) are classes that help structure language model responses. We can use the parsers to parse the extracted information to an other types such as dictionary or even a custom class. In the following we use `PydanticOutputParser` in langchain library which allows users to specify an arbitrary JSON schema and query LLMs for JSON outputs that conform to that schema. 

In [ ]:
from typing import List, Optional, Sequence
from pydantic import BaseModel, validator
from datetime import datetime
from langchain.output_parsers import PydanticOutputParser


# Updated Pydantic models to handle order_date as a string and convert it to datetime
class OrderItem(BaseModel):
    item_name: Optional[str] = None
    item_quantity: Optional[int] = None


class Order(BaseModel):
    name: str
    location: str
    order_date: datetime
    order_total: int
    order_items: List[OrderItem]

    # Custom validator to parse the order_date string to a datetime object
    @validator("order_date", pre=True)
    def parse_order_date(cls, value):
        return datetime.strptime(value, "%Y-%m-%d")


class OrderList(BaseModel):
    orders: Sequence[Order]


parser = PydanticOutputParser(pydantic_object=OrderList)
order_list = parser.parse(json_str)
print(order_list.orders)